# Analysis of Toronto neighborhoods using Foursquare API

by Tom Fenske

### Importing Libraries and dataset

In [1]:
#Importing necessary libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import folium
import json 
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print('Libraries imported!')

Libraries imported!


In [2]:
#Reading the .csv with the Toronto neighborhoods

file_path='../../II_Datasets/toronto_w_latlon.csv'
df_neigh=pd.read_csv(file_path)
df_neigh.drop('Unnamed: 0', axis=1, inplace=True)
df_neigh.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [3]:
df_neigh.shape

(99, 5)

### Creating Foursquare API call to retrieve venues around the neighbourhoods

In [4]:
#Defining Foursquare API credentials
CLIENT_ID = 'LEU5OIAP31MI5JXX1M4LGWJNMMPHZWTKHQD3W5A34FGSDPY0' # Foursquare ID
CLIENT_SECRET = '0EKHGNCNUD014O4USGZ03ZDFXQQEDNEHOMP0OJQSGBIDAMEY' # Foursquare Secret
VERSION = '20200719' # API version
LIMIT=50

In [5]:
#Defining API call for nearby venues around the coordinates of each neighbourhood

def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Calling API and preparing resulting dataset for applications of kmeans algorithm

In [6]:
#Calling the API with the Toronto dataframe

toronto_venues = getNearbyVenues(names=df_neigh['Neighbourhood'],
                                   latitudes=df_neigh['Latitude'],
                                   longitudes=df_neigh['Longitude']
                                  )



Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
B

In [7]:
print(toronto_venues.shape)
toronto_venues.head()

(4351, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
2,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
3,Parkwoods,43.753259,-79.329656,LCBO,43.757774,-79.314257,Liquor Store
4,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café


In [8]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 299 uniques categories.


In [9]:
# Applying One-Hot-Encoding in preparation of kmeans method
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Steakhouse,Storage Facility,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Syrian Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Cou

In [10]:
toronto_onehot.shape

(4351, 300)

In [11]:
#Grouping the venues of each neigbourhood
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Steakhouse,Storage Facility,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Syrian Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Cou

In [12]:
toronto_grouped.shape

(99, 300)

In [13]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Gym / Fitness Center,Caribbean Restaurant,Coffee Shop,Bakery,Breakfast Spot,Seafood Restaurant,Dim Sum Restaurant,Sri Lankan Restaurant
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Toy / Game Store,Light Rail Station,Grocery Store,Café,Bank,Park,Burger Joint,Smoothie Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Park,Coffee Shop,Pizza Place,Gas Station,Bank,Deli / Bodega,Mobile Phone Shop,Baseball Field,Sushi Restaurant,Supermarket
3,Bayview Village,Bank,Gas Station,Park,Trail,Japanese Restaurant,Grocery Store,Athletics & Sports,Café,Chinese Restaurant,Dog Run
4,"Bedford Park, Lawrence Manor East",Bakery,Sushi Restaurant,Bagel Shop,Italian Restaurant,Coffee Shop,Pub,Asian Restaurant,Café,Fast Food Restaurant,Deli / Bodega


### Applying kmeans algorithm to cluster the neighbourhoods reagrding their sorrounding venues

In [29]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:200] 

array([3, 1, 4, 4, 1, 1, 4, 1, 4, 4, 1, 1, 3, 1, 1, 1, 3, 4, 1, 1, 1, 3,
       1, 3, 1, 1, 1, 3, 3, 1, 1, 1, 3, 3, 3, 1, 1, 3, 4, 1, 1, 4, 3, 3,
       1, 3, 1, 1, 1, 1, 0, 3, 1, 4, 4, 3, 1, 1, 3, 1, 1, 3, 3, 1, 1, 1,
       4, 1, 4, 1, 3, 3, 3, 1, 1, 3, 1, 1, 4, 1, 1, 1, 1, 3, 1, 1, 2, 3,
       4, 3, 3, 3, 1, 1, 3, 3, 3, 3, 3])

In [30]:
# Add Clustr values to dataframe
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.dropna()
toronto_merged = df_neigh

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() 

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Coffee Shop,Pharmacy,Intersection,Gas Station,Supermarket,Bank,Bus Stop,Shop & Service,Café,Caribbean Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,3,Coffee Shop,Fast Food Restaurant,Middle Eastern Restaurant,Gym,Gym / Fitness Center,Shoe Store,Grocery Store,Martial Arts Dojo,Mediterranean Restaurant,Pharmacy
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Pub,Farmers Market,Restaurant,Italian Restaurant,Performing Arts Venue,Bakery,Café,Diner,Park
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Furniture / Home Store,Greek Restaurant,Cosmetics Shop,Restaurant,Dessert Shop,Fried Chicken Joint,Boutique,Kitchen Supply Store,Movie Theater
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Pizza Place,Park,Restaurant,Gastropub,Jewelry Store,Juice Bar,Sushi Restaurant,Supermarket,Beer Bar


### Creating a folium map of Toronto with coloured neighbourhoods depending on their cluster value

In [31]:
# create map
toronto_lat=43.651070
toronto_lon=-79.347015

map_clusters = folium.Map(location=[toronto_lat, toronto_lon], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ': Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [32]:
#business/commercial neighbourhood
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,0,Zoo Exhibit,Fast Food Restaurant,Pizza Place,Restaurant,Liquor Store,Supermarket,Big Box Store,Movie Theater,Financial or Legal Service,Business Service


In [33]:
#business/touristic
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1,Coffee Shop,Pub,Farmers Market,Restaurant,Italian Restaurant,Performing Arts Venue,Bakery,Café,Diner,Park
3,North York,1,Clothing Store,Furniture / Home Store,Greek Restaurant,Cosmetics Shop,Restaurant,Dessert Shop,Fried Chicken Joint,Boutique,Kitchen Supply Store,Movie Theater
4,Downtown Toronto,1,Coffee Shop,Pizza Place,Park,Restaurant,Gastropub,Jewelry Store,Juice Bar,Sushi Restaurant,Supermarket,Beer Bar
7,North York,1,Bank,Japanese Restaurant,Supermarket,Italian Restaurant,Burger Joint,Middle Eastern Restaurant,Café,Coffee Shop,Pizza Place,Restaurant
9,Downtown Toronto,1,Coffee Shop,Clothing Store,Pizza Place,Plaza,Theater,Gastropub,Cosmetics Shop,Poke Place,Ramen Restaurant,Comic Shop
14,Downtown Toronto,1,Café,Restaurant,Coffee Shop,Seafood Restaurant,Clothing Store,Furniture / Home Store,Theater,Gym,Italian Restaurant,Farmers Market
15,York,1,Coffee Shop,Italian Restaurant,Caribbean Restaurant,Café,Indian Restaurant,Ice Cream Shop,BBQ Joint,Frozen Yogurt Shop,Bagel Shop,Middle Eastern Restaurant
18,East Toronto,1,Pub,Beach,Coffee Shop,Breakfast Spot,Tea Room,Park,BBQ Joint,Caribbean Restaurant,Japanese Restaurant,Nail Salon
19,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Café,Restaurant,Park,Seafood Restaurant,Hotel,Farmers Market,Gym,Beer Bar
20,York,1,Italian Restaurant,Furniture / Home Store,Bakery,Coffee Shop,Bank,Mexican Restaurant,Pharmacy,Grocery Store,Sandwich Place,Caribbean Restaurant


In [ ]:
# function create cluster(k) and function creating map(cluster) for different k's

In [34]:
#outlier (rural area)
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Scarborough,2,National Park,Farm,Zoo Exhibit,Field,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant


In [35]:
#residential
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3,Coffee Shop,Pharmacy,Intersection,Gas Station,Supermarket,Bank,Bus Stop,Shop & Service,Café,Caribbean Restaurant
1,North York,3,Coffee Shop,Fast Food Restaurant,Middle Eastern Restaurant,Gym,Gym / Fitness Center,Shoe Store,Grocery Store,Martial Arts Dojo,Mediterranean Restaurant,Pharmacy
5,Etobicoke,3,Shopping Mall,Bank,Pharmacy,Skating Rink,Golf Course,Café,Grocery Store,Supermarket,Restaurant,Bus Stop
8,East York,3,Pharmacy,Pizza Place,Fast Food Restaurant,Park,Brewery,Gym / Fitness Center,Soccer Stadium,Gastropub,Martial Arts Dojo,Mexican Restaurant
10,North York,3,Coffee Shop,Bank,Fast Food Restaurant,Grocery Store,Caribbean Restaurant,Sandwich Place,Fried Chicken Joint,Gas Station,Restaurant,Pizza Place
13,East York,3,Coffee Shop,Pizza Place,Park,Sandwich Place,Gastropub,Breakfast Spot,Thai Restaurant,Bank,Pharmacy,Ice Cream Shop
16,Etobicoke,3,Baseball Field,Coffee Shop,Convenience Store,Bank,Grocery Store,Pizza Place,Sandwich Place,Shopping Mall,Garden,Pharmacy
17,Scarborough,3,Pizza Place,Breakfast Spot,Fast Food Restaurant,Bank,Coffee Shop,Sandwich Place,Discount Store,Restaurant,Liquor Store,Beer Store
21,Scarborough,3,Coffee Shop,Fast Food Restaurant,Sandwich Place,Pharmacy,Bank,Supermarket,Pizza Place,Grocery Store,Bar,Chinese Restaurant
25,Scarborough,3,Coffee Shop,Indian Restaurant,Clothing Store,Sandwich Place,Gas Station,Restaurant,Bank,Fried Chicken Joint,Bar,Sushi Restaurant


In [36]:
#touristic
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,4,Park,Sandwich Place,Restaurant,Farmers Market,Electronics Store,Bank,Theater,Grocery Store,Gym,Gym / Fitness Center
12,Scarborough,4,Park,Grocery Store,Breakfast Spot,Gym,Gym / Fitness Center,Neighborhood,Burger Joint,Italian Restaurant,Farmers Market,Ethiopian Restaurant
27,North York,4,Park,Coffee Shop,Pizza Place,Gas Station,Bank,Deli / Bodega,Mobile Phone Shop,Baseball Field,Sushi Restaurant,Supermarket
38,North York,4,Bank,Gas Station,Park,Trail,Japanese Restaurant,Grocery Store,Athletics & Sports,Café,Chinese Restaurant,Dog Run
45,East Toronto,4,Park,Brewery,Indian Restaurant,Café,Beach,Bakery,Bar,Coffee Shop,Burrito Place,BBQ Joint
48,North York,4,Park,Bakery,Skating Rink,Asian Restaurant,Pharmacy,Mexican Restaurant,Vietnamese Restaurant,Bank,Café,Italian Restaurant
49,Scarborough,4,Park,Harbor / Marina,Pizza Place,Beach,Discount Store,Coffee Shop,Pub,Grocery Store,Sandwich Place,Ice Cream Shop
55,Scarborough,4,Park,Thai Restaurant,Restaurant,Golf Course,Café,Gym,Gym Pool,Skating Rink,Diner,General Entertainment
79,Central Toronto,4,Italian Restaurant,Park,Sushi Restaurant,Coffee Shop,Grocery Store,Café,Gym,Spa,Liquor Store,Supermarket
82,Central Toronto,4,Italian Restaurant,Café,Park,Sushi Restaurant,Spa,Liquor Store,Bagel Shop,BBQ Joint,Electronics Store,Supermarket
